# Find My Son

## Load Libraries and set mode(Local or GLab)

In [195]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
%matplotlib inline

#mode:'local' means on local machine, 'glabo' means on google colab 
mode = 'local'
#mode = 'glabo'

if mode =='glabo':
    ROOT_DIR='/content/gdrive/My Drive/opencv'
    cascade_path = '/./usr/local/lib/python3.6/dist-packages/cv2/data/'
    OUT_DIR = os.path.join('./')
elif mode:
    ROOT_DIR='./'
    cascade_path="C:\\Users\\xkogi\\.conda\\envs\\myenv\\Lib\\site-packages\\cv2\\data\\"
    OUT_DIR = os.path.join('./','output')
    
# alias
scale_factor=1.1 #default = 1.1
min_size = (5,5)
min_neighbour=3 #default=3
(resize_width, resize_height) = (130, 100)

MOV_FULL_L='gassou_full_large.mp4'
MOV_FULL_S='gassou_full_small.mp4'
MOV_LONG_L ='gassou_30sec_large.mp4'
MOV_LONG_S ='gassou_30sec_small.mp4'
MOV_SHORT_L = 'gassou_15sec_large.mp4'
MOV_SHORT_S = 'gassou_15sec_small.mp4'
IMG_REPORT = 'img_report.jpg'
IMG_INTEVIEW = 'img_interviewMovie.jpg'

# movie file
def getImageFile(name=''):
    path = os.path.join(ROOT_DIR,name)
    if os.path.exists(os.path.join(path)):
        return path
    else:
        return 0
    
# get haar cascade file
def getcascade(name='frontalface_alt2'):
    if name == 'eye':
        return os.path.join(cascade_path,'haarcascade_eye.xml')
    elif name == 'eye_tree_eyeglasses':
        return os.path.join(cascade_path,'haarcascade_eye_tree_eyeglasses.xml')
    elif name == 'frontalcatface':
        return os.path.join(cascade_path,'haarcascade_frontalcatface.xml')
    elif name == 'frontalcatface_extended':
        return os.path.join(cascade_path,'haarcascade_frontalcatface_extended.xml')
    elif name == 'frontalface_alt':
        return os.path.join(cascade_path,'haarcascade_frontalface_alt.xml')
    elif name == 'frontalface_alt2':
        return os.path.join(cascade_path,'haarcascade_frontalface_alt2.xml')
    elif name == 'frontalface_alt_tree':
        return os.path.join(cascade_path,'haarcascade_frontalface_alt_tree.xml')
    elif name == 'frontalface_default':
        return os.path.join(cascade_path,'haarcascade_frontalface_default.xml')
    elif name == 'fullbody':
        return os.path.join(cascade_path,'haarcascade_fullbody.xml')
    elif name == 'lefteye_2splits':
        return os.path.join(cascade_path,'haarcascade_lefteye_2splits.xml')
    elif name == 'licence_plate_rus_16stages':
        return os.path.join(cascade_path,'haarcascade_licence_plate_rus_16stages.xml')
    elif name == 'lowerbody':
        return os.path.join(cascade_path,'haarcascade_lowerbody.xml')
    elif name == 'profileface':
        return os.path.join(cascade_path,'haarcascade_profileface.xml')
    elif name == 'righteye_2splits':
        return os.path.join(cascade_path,'haarcascade_righteye_2splits.xml')
    elif name == 'russian_plate_number':
        return os.path.join(cascade_path,'haarcascade_russian_plate_number.xml')
    elif name == 'smile':
        return os.path.join(cascade_path,'haarcascade_smile.xml')
    elif name == 'upperbody':
        return os.path.join(cascade_path,'haarcascade_upperbody.xml')
    else:
        return os.path.join(cascade_path,'haarcascade_frontalface_default.xml')
    
# format time(second) as HH:MM:ss.mmm
def format_time(sec):
    msec = (int)(sec * 1000.)
    s, ms = divmod(msec, 1000)
    m, s =divmod(s,60)
    return '{:02d}:{:02d}.{:03d}'.format(m,s,ms)

def format_time_forfile(sec):
    msec = (int)(sec * 1000.)
    s, ms = divmod(msec, 1000)
    m, s =divmod(s,60)
    return '{:02d}m{:02d}s{:03d}ms'.format(m,s,ms)

print(getcascade('frontalface_alt'))
print(format_time_forfile(182.3))

C:\Users\xkogi\.conda\envs\myenv\Lib\site-packages\cv2\data\haarcascade_frontalface_alt.xml
03m02s300ms


In [196]:
# create capture and get params
def getCapture(movie,cascade=''):
    cap = cv2.VideoCapture(getImageFile(movie))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    face_cascade=cv2.CascadeClassifier(getcascade(cascade))
    print('fps',fps)
    return cap,width,height,fps,face_cascade

def getInterval(fps):
    interval =1.0/fps
    time_wait = (int)(interval * 1000.0)
    print('interval: ',interval,'time_wait: ',time_wait)
    return interval,time_wait

# detect face from movie and show it on screen
def face_detect_mov(movie,cascade):
    cap,width,height,fps,face_cascade = getCapture(movie,cascade)
    interval,time_wait = getInterval(fps)
    ret, frame = cap.read()
    FLAG_FACE = True
    time_stamp=0.0
    font = cv2.FONT_HERSHEY_PLAIN
    while(1):
        ret, frame = cap.read()
        if ret == True:
            if FLAG_FACE == True:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, scaleFactor=scale_factor, minNeighbors=min_neighbour, minSize=min_size) # , 1.3, 5)
                #img = np.zeros((width,height,3), np.uint8)
                for(x, y, w, h) in faces:
                    frame = cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
                    #img[x:x+w,y:y+h]=frame[x:x+h,y:y+h]
            time_stamp = time_stamp + interval
            cv2.putText(frame,format_time(time_stamp),(10,50),font,4,(255,255,255),2,cv2.LINE_AA)
            cv2.imshow('frame',frame)
            k = cv2.waitKey(time_wait) & 0xff
            if k == 27:
                break;
        else:
            break

    cv2.destroyAllWindows()
    cap.release()
    
face_detect_mov(MOV_LONG_S,'')

fps 23.976023976023978
interval:  0.04170833333333333 time_wait:  41


In [197]:
# detect facce from movie, write rectangle and save as movie
# this is not complete, I don't want run here, so just changed as a function
def face_detect_mov_save(movie,output,cascade=''):
    cap,width,height,fps,face_cascade = getCapture(movie,cascade)
    interval,time_wait = getInterval(fps)

    time_stamp=0.0
    font = cv2.FONT_HERSHEY_PLAIN

    # VideoWriter を作成する。
    if mode == 'glabo':
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    else:
        fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    writer = cv2.VideoWriter(os.path.join(OUT_DIR,output), fourcc, fps, (width, height))

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=scale_factor, minNeighbors=min_neighbour, minSize=min_size) # , 1.3, 5)
            for(x, y, w, h) in faces:
                frame = cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
            time_stamp = time_stamp + interval
            cv2.putText(frame,format_time(time_stamp),(10,50),font,4,(255,255,255),2,cv2.LINE_AA)
            writer.write(frame)
        else:
            break
        
    cap.release()
    writer.release()
#face_detect_mov_save(MOV_LONG_S,'out_move_long.avi')

fps 23.976023976023978
interval:  0.04170833333333333 time_wait:  41


## Face Recognition
* see [https://github.com/informramiz/opencv-face-recognition-python](https://github.com/informramiz/opencv-face-recognition-python)

In [198]:
# function to detect face using openCV
subjects = ["","Son","boy","girl","suzu",'shishamo','family']
def detect_face(img):
    #convert the test image to gray image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # load OpenCV face detector
    face_cascade =cv2.CascadeClassifier(getcascade(''))

    faces = face_cascade.detectMultiScale(gray, scaleFactor=scale_factor, minNeighbors=min_neighbour, minSize=min_size) # , 1.3, 5)

    if(len(faces) == 0):
        return None, None
    
    (x,y,w,h) = faces[0]
    
    return gray[y:y+w, x:x+h], faces[0]
    
# prepare training data
# this function will read all persons training images, detect face from each image
# and will return two lists of exactly same size,
# one list of face and another list of labels for each face
def prepare_training_data(data_folder_path):
    # ---step 1---
    # get the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)
    
    # list to hold all subject faces
    faces = []
    # list to hold labels for all subjects
    labels = []
    
    # let's go through each directory and read images within it
    for dir_name in dirs:
        # our subjcect directories start with letter 's' so ignore any non-relevant directoies if any
        if not dir_name.startswith("s"):
            continue
        print('dir: ',dir_name)
        # --step 2---
        # extract label number of subjet from dir_name
        # format of dir name = slabel
        label = int(dir_name.replace("s",""))
        
        # build path of directory containing image for current subject
        # sample subject_dir_path = "training-data/s1"
        subject_dir_path =os.path.join(data_folder_path,dir_name)
        
        # get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
        
        # ---step 3---
        # go through each image name, read image,
        # detect face and add face to list of faces
        for image_name in subject_images_names:
            # ignore system files like .DS_Store
            if image_name.startswith("."):
                continue
                
            # build image path
            image_path = os.path.join(subject_dir_path,image_name)
            
            # read image
            image = cv2.imread(image_path)
            
            #display an image window to show the image
            cv2.imshow("training on image...",image)
            cv2.waitKey(100)
            
            # detect face
            face, rect = detect_face(image)
            
            # ---step 4---
            # for the purpose of this, we will ignore faces that are not detected
            if face is not None:
                #print(image_name,'detect')
                # add face to list of faces
                face_resize = cv2.resize(face, (resize_width,resize_height))
                cv2.imshow("training on image...",face_resize)
                faces.append(face_resize)
                cv2.waitKey(200)
                # add label for this face
                labels.append(label)
            else:
                print(image_name,'face not detected.')
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
        
    return faces, labels

In [199]:
print("preparing data...")
faces, labels = prepare_training_data("training-data")
print("data prepared")

print("Total faces: ", len(faces))
print("Total labels; ",len(labels))

preparing data...
dir:  s1
dir:  s2
dir:  s3
dir:  s4
dir:  s5
dir:  s6
data prepared
Total faces:  74
Total labels;  74


In [200]:
file_name='./training-data/s6/7.jpg'
# file_name='../image-collector-master/shishamo/23.jpg'
if os.path.exists(file_name):
    img=cv2.imread(file_name)
    face,rect=detect_face(img)
    if face is not None:
        (x,y,w,h)=rect
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.imshow('test',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
else:
    print(file_name,'not exits.')

./training-data/s6/7.jpg not exits.


In [211]:
# create face recognizer
#face_recognizer = cv2.face.LBPHFaceRecognizer_create()
#face_recognizer = cv2.face.createLBPHFaceRecognizer()
face_recognizer = cv2.face_LBPHFaceRecognizer.create()
#or use EigenFaceRecognizer by replacing above line with 
#face_recognizer = cv2.face.createEigenFaceRecognizer()
#face_recognizer = cv2.face.EigenFaceRecognizer_create()

#or use FisherFaceRecognizer by replacing above line with 
#face_recognizer = cv2.face.FisherFaceRecognizer_create()
(width, height) = (130, 100)

# train face recognizer
face_recognizer.train(faces,np.array(labels))

In [212]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
    
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x,y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0,255,0), 2)
    

In [213]:
# recognize!
def predict(test_image):
    img = test_img.copy()
    face, rect = detect_face(img)
    
    #prediction
    face_resize = cv2.resize(face, (resize_width, resize_height))
    label = face_recognizer.predict(face_resize)
    print(label)
    label_text = subjects[label[0]]
    
    draw_rectangle(img, rect)
    draw_text(img, label_text, rect[0], rect[1]-5)
    return img

def predict_son_only(test_image,face_cascade):
    img = test_image.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=scale_factor, minNeighbors=min_neighbour, minSize=min_size) # , 1.3, 5)
    found = False
    for(x, y, w, h) in faces:
        face = gray[y:y+w, x:x+h]
        rect = (x,y,w,h)
        face_resize = cv2.resize(face, (width, height))
        label = face_recognizer.predict(face_resize)
        if label[0] == 1:
            print(subjects[label[0]],label)
            label_text = subjects[label[0]] #+'('+str(label[1])+')'
            found = True
            draw_rectangle(img, rect)
            draw_text(img, label_text, rect[0], rect[1]-5)
    return found,img

def predictall(test_image,face_cascade,disp=False):
    img = test_image.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      # load OpenCV face detector
#    face_cascade =cv2.CascadeClassifier(getcascade(''))
    faces = face_cascade.detectMultiScale(gray, scaleFactor=scale_factor, minNeighbors=min_neighbour, minSize=min_size) # , 1.3, 5)
    found = False
    for(x, y, w, h) in faces:
        face = gray[y:y+w, x:x+h]
        rect = (x,y,w,h)
        draw_rectangle(img, rect)
        face_resize = cv2.resize(face, (width, height))
        label = face_recognizer.predict(face_resize)
        if disp:
            print(subjects[label[0]],label)
        if label[1] < 500:
            label_text = subjects[label[0]] #+'('+str(label[1])+')'
            found = found or label[0] == 1
            draw_text(img, label_text, rect[0], rect[1]-5)
    return found,img


In [214]:
test_img = cv2.imread("./test-data/12.jpg")
face_cascade =cv2.CascadeClassifier(getcascade(''))
found,predict_img = predictall(test_img,face_cascade,True)
#found,predict_img = predict_son_only(test_img,face_cascade)
if found:
    print("found!")
cv2.imshow("test",predict_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


girl (3, 102.77895658475059)
boy (2, 103.07513575520454)
family (6, 100.27964613958467)
Son (1, 111.10979660731897)
family (6, 87.11710511905338)
boy (2, 94.97964196463933)
Son (1, 102.647247632844)
family (6, 99.82292955329477)
girl (3, 88.8998334025174)
boy (2, 108.10849404729298)
found!


In [215]:
# face recognize on movie
def face_recognize_mov(movie,outpath=None,cascade='',disp=True):
    cap,width,height,fps,face_cascade = getCapture(movie,cascade)
    interval,time_wait = getInterval(fps)
    time_stamp=0.0
    font = cv2.FONT_HERSHEY_PLAIN
    counter = 0
    while(1):
        ret, frame = cap.read()
        if ret == True:
            found,frame = predictall(frame,face_cascade)
            time_stamp = time_stamp + interval
            strTime = format_time(time_stamp)
            cv2.putText(frame,strTime,(10,50),font,4,(255,255,255),2,cv2.LINE_AA)
            if disp:
                cv2.imshow('frame',frame)
            if found:
                counter = counter + 1
                print(str(counter),'found at:',strTime)
                if outpath is not None:
                    filename=os.path.join(outpath,format_time_forfile(time_stamp)+'.png')
                    cv2.imwrite(filename,frame)
            if disp:
                k = cv2.waitKey(time_wait) & 0xff
                if k == 27:
                    break;
        else:
            break

    cv2.destroyAllWindows()
    cap.release()
face_recognize_mov(MOV_FULL_S)

fps 23.976023976023978
interval:  0.04170833333333333 time_wait:  41
1 found at: 00:00.166
2 found at: 00:00.208
3 found at: 00:00.250
4 found at: 00:00.291
5 found at: 00:00.333
6 found at: 00:00.375
7 found at: 00:00.917
8 found at: 00:01.001
9 found at: 00:01.126
10 found at: 00:01.376
11 found at: 00:01.626
12 found at: 00:01.876
13 found at: 00:02.085
14 found at: 00:02.210
15 found at: 00:02.585
16 found at: 00:02.627
17 found at: 00:02.669
18 found at: 00:02.711
19 found at: 00:02.794
20 found at: 00:02.836
21 found at: 00:02.877
22 found at: 00:03.294
23 found at: 00:06.297
24 found at: 00:06.423
